In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import LeaveOneOut,cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import seaborn as sns
from sklearn.kernel_approximation import PolynomialCountSketch
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

In [2]:
#generate and filter the data
data = pd.read_csv('new_data.csv')
data['Sex'] = data['Sex'].replace('M',1)
data['Sex'] = data['Sex'].replace('F',0)
data['Side'] = data['Side'].replace('L',0)
data['Side'] = data['Side'].replace('R',1)
X = data[['Age','Sex','BMI','GX AT VO2 (mL/kg/min)','GX AT VE/VCO2','GX VO2 Max VO2 (mL/kg/min)','GX VO2 Max VO2/Pred (%)','GX VO2 Max VO2/HR (mL/beat)','GX Predicted VO2/HR (mL/beat)','GX Work Max Work (Watts)','GX Predicted Work (Watts)','PF Pre FEV1/FVC (%)','PF Pre FEV1 (L)','Side','pred mort','LOS']]
X = X.dropna(axis=0)

#separate X,y
X1 = X[['Age','Sex','BMI','GX AT VO2 (mL/kg/min)','GX AT VE/VCO2','GX VO2 Max VO2 (mL/kg/min)','GX VO2 Max VO2/Pred (%)','GX VO2 Max VO2/HR (mL/beat)','GX Predicted VO2/HR (mL/beat)','GX Work Max Work (Watts)','GX Predicted Work (Watts)','PF Pre FEV1/FVC (%)','PF Pre FEV1 (L)','Side','pred mort']]
y = X[['LOS']]

In [11]:
loo = LeaveOneOut()
lg = LinearRegression()
y = np.array(y)
X1 = np.array(X1)
mean_ab_error = []
mean_sq_error = []
median_error = []
pred = []
y_true = []
for train_index, test_index in loo.split(X1):
    X_train, X_test = X1[train_index], X1[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lg.fit(X_train,y_train)
    
    predict_data = lg.predict(X_test)
    if(mean_sq_error(y_test,predict_data) < 500):
        pred.append(predict_data)
        y_true.append(y_test)
        mean_ab_error.append(mean_absolute_error(y_test,predict_data))
        mean_sq_error.append(mean_squared_error(y_test,predict_data))
        median_error.append(median_absolute_error(y_test,predict_data))
    
print("Leave one out validation")
MAE = np.sum(np.array(mean_ab_error)) / len(mean_ab_error)
print(f"MAE of the model is {MAE}")
MSE = np.sum(np.array(mean_sq_error)) / len(mean_sq_error)
print(f"MSE of the model is {MSE}")
Median = np.median(np.array(median_error))
print(f"Median absolute error of the model is {Median}")
SD = np.std(pred)
print(f"Standard Deviation of the model is {SD}")
# r2 = r2_score(y_true,pred)
# print(f"R2 score of the model is {r2}")
y_new = np.array(y_true).reshape(-1)
pred_new = np.array(pred).reshape(-1)
r2 = r2_score(y_new,pred_new)
print(f"R2 score of the model is {r2}")
mean_sq_error = np.array(mean_sq_error)
mean_sq_error = mean_sq_error[mean_sq_error > 500]
mean_sq_error

Leave one out validation
MAE of the model is 5.135661736283378
MSE of the model is 80.78912831541813
Median absolute error of the model is 3.4795368050276956
Standard Deviation of the model is 2.8454930781008962
R2 score of the model is -0.04063835879262312


array([ 594.55880404, 1203.87163448, 7055.5213127 ,  737.7526126 ,
       1327.87237652,  526.94384812])